In [1]:
# Regression using ANN in PyTorch

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [4]:
# Load the dataset
data = pd.read_csv("Salary_Data.csv")
data.dropna(inplace=True)

In [5]:
# Features and label
X = data[['YearsExperience']].values  # input feature
y = data['Salary'].values  # target variable

In [6]:
# Feature Scaling
scaler_X = StandardScaler()
scaler_y = StandardScaler()

In [7]:
X = scaler_X.fit_transform(X)
y = scaler_y.fit_transform(y.reshape(-1, 1))  # Reshape y for scaling

In [8]:
# Convert to tensors
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

In [9]:
# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:

# Send data to device
X_train, X_test = X_train.to(device), X_test.to(device)
y_train, y_test = y_train.to(device), y_test.to(device)

In [ ]:
# Define a simple ANN model for regression
class RegressionANN(nn.Module):
    def __init__(self):
        super(RegressionANN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(1, 10),
            nn.ReLU(),
            nn.Linear(10, 5),
            nn.ReLU(),
            nn.Linear(5, 1)
            
            # No output function because we want to take the output as it is without modification of the linear function
        )

    def forward(self, x):
        return self.model(x)

In [16]:
# Instantiate model, loss function and optimizer
model = RegressionANN().to(device)
criterion = nn.MSELoss()  # Mean Squared Error for regression
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [17]:
# Training loop
n_epochs = 100
for epoch in range(n_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

    # Calculate training R2 Score
    with torch.no_grad():
        train_preds = scaler_y.inverse_transform(outputs.cpu().numpy())
        y_train_actual = scaler_y.inverse_transform(y_train.cpu().numpy())
        train_r2 = r2_score(y_train_actual, train_preds)

    # Evaluation on validation set
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_test)
        val_loss = criterion(val_outputs, y_test)

        val_preds = scaler_y.inverse_transform(val_outputs.cpu().numpy())
        y_val_actual = scaler_y.inverse_transform(y_test.cpu().numpy())
        val_r2 = r2_score(y_val_actual, val_preds)

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{n_epochs}], Train Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}, Train R2: {train_r2:.4f}, Val R2: {val_r2:.4f}")


Epoch [10/100], Train Loss: 0.6674, Val Loss: 0.4436, Train R2: 0.3656, Val R2: 0.3691
Epoch [20/100], Train Loss: 0.2173, Val Loss: 0.1019, Train R2: 0.7935, Val R2: 0.8551
Epoch [30/100], Train Loss: 0.0368, Val Loss: 0.0795, Train R2: 0.9650, Val R2: 0.8869
Epoch [40/100], Train Loss: 0.0508, Val Loss: 0.1278, Train R2: 0.9517, Val R2: 0.8182
Epoch [50/100], Train Loss: 0.0273, Val Loss: 0.0675, Train R2: 0.9740, Val R2: 0.9040
Epoch [60/100], Train Loss: 0.0273, Val Loss: 0.0618, Train R2: 0.9741, Val R2: 0.9121
Epoch [70/100], Train Loss: 0.0237, Val Loss: 0.0770, Train R2: 0.9775, Val R2: 0.8904
Epoch [80/100], Train Loss: 0.0230, Val Loss: 0.0701, Train R2: 0.9781, Val R2: 0.9003
Epoch [90/100], Train Loss: 0.0226, Val Loss: 0.0682, Train R2: 0.9785, Val R2: 0.9030
Epoch [100/100], Train Loss: 0.0221, Val Loss: 0.0716, Train R2: 0.9790, Val R2: 0.8982


In [18]:
# Final evaluation
model.eval()
with torch.no_grad():
    predictions = model(X_test)
    predictions = predictions.cpu().numpy()
    y_test_np = y_test.cpu().numpy()

    # Inverse scaling
    predictions = scaler_y.inverse_transform(predictions)
    y_actual = scaler_y.inverse_transform(y_test_np)

    mse = mean_squared_error(y_actual, predictions)
    r2 = r2_score(y_actual, predictions)

In [19]:
print("\nTest MSE:", mse)
print("R2 Score:", r2)


Test MSE: 51999216.0
R2 Score: 0.8981996178627014
